# __*Segmenting and Clustering Neighborhoods in Toronto*__

 Author : Abhijeet Prasad

In [1]:
# Install bs4 library
!pip install bs4

In [2]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

## __Importing Data__

In [3]:
# get the contents of the webpage in text format and store in a variable called data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text

In [4]:
soup = BeautifulSoup(data,"html5lib")

In [5]:
#find a html table in the web page
tables = soup.find_all('table')
len(tables) # Total Number of tables in the file

3

In [6]:
neighborhood_data = pd.DataFrame(columns=["Postal Code", "Borough", "Neighbourhood"])

for row in tables[0].findAll('td'):      # tables[0] contains the data we are intrested in.
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        Postal_Code = row.p.text[:3]
        borough = (row.span.text).split('(')[0]
        neighborhood = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        neighborhood_data = neighborhood_data.append({"Postal Code":Postal_Code, "Borough":borough, "Neighbourhood":neighborhood}, ignore_index=True)

neighborhood_data['Borough']=neighborhood_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

neighborhood_data

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
neighborhood_data.shape

(103, 3)

## __Getting Latitude and Longitude coordinates of each neighborhood__

In [8]:
# install gecoder library
!pip install geocoder

__The Below Code doesn't work__

__Alternate Method- Using the CSV file Provided__

In [11]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### __Adding Latitude and Longitude to neighborhood_data

In [14]:
coordinates.shape

(103, 3)

In [19]:
# Since the dimensions are the same, we can try to join on the postal codes to get the required data.

neighborhood_data['Latitude']=coordinates['Latitude']
neighborhood_data['Longitude']=coordinates['Longitude']

neighborhood_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
100,M7Y,East Toronto Business,Enclave of M4L,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437
